In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
#         validation_split=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        "/content/drive/My Drive/BinaryReID/Data/train",
        target_size=(120, 160),
        batch_size=32,
        class_mode='binary',
        shuffle=True)

validation_generator = test_datagen.flow_from_directory(
        "/content/drive/My Drive/BinaryReID/Data/test",
        target_size=(120, 160),
        batch_size=32,
        class_mode='binary',
        shuffle=False)

model_weights_path = "/content/drive/My Drive/BinaryReID/Weights/weights.hdf5"

# Specify the input shape to the first convolutional layer
input_shape = (120, 160, 3)
nClasses = 2
def createModel():
    model = Sequential()
    # a convolution layer of 32 features of size 3x3 with relu activation and zero padding
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    # a convolution layer of 32 features of size 3x3 with relu activation
    model.add(Conv2D(32, (3, 3), activation='relu'))
    # a batch normalization layer
    model.add(BatchNormalization())
    # maxpooling layer of filter size 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # a convolution layer of 64 features of size 3x3 with relu activation and zero padding
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    # a convolution layer of 64 features of size 3x3 with relu activation
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # a batch normalization layer
    model.add(BatchNormalization())
    # maxpooling layer of filter size 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # a convolution layer of 64 features of size 3x3 with relu activation and zero padding
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    # a convolution layer of 64 features of size 3x3 with relu activation
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # a batch normalization layer
    model.add(BatchNormalization())
    # maxpooling layer of filter size 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # a dropout layer of 50%
    model.add(Dropout(0.5))
    
    # flatten the output of the previous layer
    model.add(Flatten())
    # add a dense layer that outputs 512 units and apply relu activation
    model.add(Dense(512, activation='relu'))
    # a dropout layer of 50%
    model.add(Dropout(0.5))
    # add a dense layer with a softmax activation to classify the images
    model.add(Dense(nClasses, activation='softmax'))
     
    return model

# model1_weights_path = "/content/drive/My Drive/BinaryReID/Weights/weights.hdf5"
 
model1 = createModel()
model1.load_weights("/content/drive/My Drive/BinaryReID/Weights/weights.hdf5")
model1.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ----------------- 
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size 

# ---------------------

#Justification for step_size: To pass over the whole data per epoch

# ---------------------

filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model1.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=40, callbacks=callbacks_list
)

Using TensorFlow backend.


Found 3872 images belonging to 2 classes.
Found 860 images belonging to 2 classes.
Epoch 1/40
121/121 [==============================] - 1148s 9s/step - loss: 0.9123 - acc: 0.7404 - val_loss: 1.0794 - val_acc: 0.7488

Epoch 00001: val_acc improved from -inf to 0.74880, saving model to weights.best.hdf5
Epoch 2/40
121/121 [==============================] - 34s 284ms/step - loss: 0.6288 - acc: 0.7978 - val_loss: 6.8914 - val_acc: 0.5483

Epoch 00002: val_acc did not improve from 0.74880
Epoch 3/40
121/121 [==============================] - 35s 292ms/step - loss: 0.6284 - acc: 0.8182 - val_loss: 0.9140 - val_acc: 0.8176

Epoch 00003: val_acc improved from 0.74880 to 0.81763, saving model to weights.best.hdf5
Epoch 4/40
121/121 [==============================] - 35s 289ms/step - loss: 0.4918 - acc: 0.8280 - val_loss: 0.3718 - val_acc: 0.8925

Epoch 00004: val_acc improved from 0.81763 to 0.89251, saving model to weights.best.hdf5
Epoch 5/40
121/121 [==============================] - 35s 28